# Make freeview images

- make freeview images, adding angle jitter  
- several jitters per subject
- currently settings: left hemisphere, ventral view, inflated (fsaverage) base, curv or thickness overlay
- expects a folder in working directory called ./data/thickness or ./data/curv

In [1]:
import numpy as np
import os
import sys

In [2]:
# specify location of data
# expected format is .mgh, in a subdirectory labeled thickness of curv
data_dir = '/Users/samjohns/data/transfer-curv-thickness/sub-NDARZZ810LVF/fsa-data'

In [3]:
os.chdir(data_dir)

In [4]:
thickness_subdir = '/thickness'
thickness_dir = data_dir + thickness_subdir

In [5]:
image_subdir = '/images/images-thickness'
image_dir = data_dir + image_subdir

if not os.path.exists(image_dir):
    os.mkdir(image_dir)

In [6]:
# default values
SCALE=10
VIEWS=10
CENTER=np.array([-65, 0, 0])

def generate_angles(nviews=VIEWS, scale=SCALE, center=CENTER):
    center = np.array(center) # cast to np.array for broadcasting addition
    angle_jitters = scale * (np.array(np.split(np.random.rand(3 * nviews), nviews)) - 0.5)
    return center + angle_jitters

In [7]:
def make_command_strings(input_file_name, pic_base_name, angles):
    command_strings = []
    for i, a in enumerate(angles):
        command_strings.append(f'freeview -f .{thickness_subdir}/lh.inflated:curvature_method="binary":overlay=.{thickness_subdir}/{input_file_name}:overlay_threshold=0,4.5 -cam dolly 1.8 elevation {a[0]:.2f} azimuth {a[1]:.2f} roll {a[2]:.2f} -ras -45 0 0 -cc -ss .{image_subdir}/{pic_base_name}-{a[0]:.2f}-{a[1]:.2f}-{a[2]:.2f}.png')
    return command_strings

In [8]:
# process input filenames; filter out other directory entries; strip .mgh extension
input_files = [f[:-4] for f in os.listdir(thickness_dir) if 'sub-' in f and '.mgh' in f]

In [9]:
input_files[:5], len(input_files)

(['sub-NDARTV296UNC.fsa.lh.thickness',
  'sub-NDARZT940RZG.fsa.lh.thickness',
  'sub-NDARWK793VK4.fsa.lh.thickness',
  'sub-NDARZR529EJ1.fsa.lh.thickness',
  'sub-NDARBN100LCD.fsa.lh.thickness'],
 2388)

In [10]:
def make_subject_images(input_file):
    angles = generate_angles()
    command_strings = make_command_strings(input_file+'.mgh', input_file, angles)
    for command_string in command_strings:
        os.system(command_string)
        
        ## debugging ##
        # print(command_string)

In [11]:
# this cell makes the images
# it can take a long time (~ 2 - 10s per image)

# resume from last time
for input_file in input_files[2095:]:
    make_subject_images(input_file)

In [ ]:
31 roll 0.71 -ras -45 0 0 -cc -ss test.png

In [ ]:
! freeview -f ./lh.inflated:curvature_method='binary':overlay=./lh.thickness -cam dolly 1.5 elevation 65 azimuth 0 roll 0 -ss test6500.png